In [43]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import gsw

from scipy.integrate import solve_ivp

from scipy.interpolate import interp1d

In [44]:
ds = xr.open_dataset('data/MBARI_StationM_ADCP_201711_201811.nc')

In [45]:
lat = 35+8.4585/60
bini = 15

Smooth ADCP time series
* 30-min running median to remove spikes
* 60-min running mean to smooth

In [46]:
ds['binheight'][bini]

<xarray.DataArray 'binheight' ()>
array(17.14)

In [47]:
ds['Eas_sm'] = ds['Eas'].rolling(time=6,center=True).median().rolling(time=12,center=True).mean()
ds['Nor_sm'] = ds['Nor'].rolling(time=6,center=True).median().rolling(time=12,center=True).mean()

In [48]:
plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Eas_sm'][:,bini]);
plt.plot(ds['time'],ds['Eas'][:,bini],'--');
plt.plot(ds['time'],ds['Eas_sm'][:,1],'-');
#plt.plot(ds['time'],ds['Eas'][:,1]);
plt.ylabel('m/s')
plt.title('eastward velocity')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'eastward velocity')

In [49]:
plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Nor_sm'][:,bini]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.title('northward velocity')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'northward velocity')

### Test with limited obs


In [50]:
zf = ds['binheight'][bini] # upper boundary height
zobs = np.array(ds['binheight'][0:(bini+1)])

In [51]:
i = 1j
kappa = 0.41 # Von Karman constant
f = gsw.f(lat)

In [52]:
gi = np.isfinite(ds['Eas_sm'][:,0])
tobs = np.array((ds['time'][gi] - ds['time'][gi][0])/np.timedelta64(1,'s')) # array of times in seconds
deltat = tobs[1]-tobs[0]

In [53]:
wobs = np.array(ds['Eas_sm'][gi,0:(bini+1)]+i*ds['Nor_sm'][gi,0:(bini+1)])
dwobsdt = np.gradient(wobs,deltat,axis=0)

In [54]:
np.where(gi)

(array([    9,    10,    11, ..., 98198, 98199, 98200]),)

In [55]:
ds['time'][gi]

<xarray.DataArray 'time' (time: 98192)>
array(['2017-11-10T10:48:13.760000000', '2017-11-10T10:53:13.760000000',
       '2017-11-10T10:58:13.760000000', ..., '2018-10-17T09:13:13.720000000',
       '2018-10-17T09:18:13.720000000', '2018-10-17T09:23:13.720000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2017-11-10T10:48:13.760000 ... 2018-10-17T09:23:13.720000

In [56]:
ds['time'][9]

<xarray.DataArray 'time' ()>
array('2017-11-10T10:48:13.760000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2017-11-10T10:48:13.760000

In [57]:
def dwdt_bbl(t, w_in):
    # Note that w_in is real and complex parts concatenates
    # need to combine into complex numbers
    # Scipy docs say that ode solvers RK45 and BDF can be used in complex plane
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.BDF.html#scipy.integrate.BDF
    # but perhaps this does not work for *arrays* of complex numbers?
    
    global zo
    
    N = int(len(w_in)/2)
    w = w_in[:N]+i*w_in[N:]
    
    winf = np.interp(t,tobs,wobs[:,bini])
    dwinfdt = np.interp(t,tobs,dwobsdt[:,bini])

    #F = dwinfdt + i*f*winf
    ustar = kappa*zo*np.abs(w[1])/dz[0]
    coef1 = 2*kappa*ustar/(dz[:-1]+dz[1:])

    wall = np.concatenate([np.atleast_1d(0+i*0),w,np.atleast_1d(winf)])
    
    st =  (coef1*(z[1:-1]/dz[1:])*wall[2:]
     - coef1*(z[1:-1]/dz[1:]+z[:-2]/dz[:-1])*wall[1:-1]
     + coef1*(z[:-2]/dz[:-1])*wall[:-2])
    
    F = dwinfdt + i*f*winf - st[-1]
    
    dwalldt = np.nan*np.ones(np.shape(wall))+i*np.nan*np.ones(np.shape(wall))
    dwalldt[1:-1] = (F +
                     coef1*(z[1:-1]/dz[1:])*wall[2:] +
                     (-i*f - coef1*(z[1:-1]/dz[1:]+z[:-2]/dz[:-1]))*wall[1:-1] +
                     coef1*(z[:-2]/dz[:-1])*wall[:-2])
    
    dwdt = dwalldt[1:-1]
    
    dwdt_out = np.concatenate([np.real(dwdt),np.imag(dwdt)])
    
    return dwdt_out

In [59]:
nzo = 7
zo_array = np.logspace(-4,-1,nzo) # roughness length array

tf = 31*86400
ntimes = len(tobs[tobs<=tf])

nz = 201

w_sol_all = np.nan*np.ones([ntimes,nz-2,nzo]) + i*np.nan*np.ones([ntimes,nz-2,nzo])
w_sol_obs_all = np.nan*np.ones([ntimes,len(zobs),nzo]) + i*np.nan*np.ones([ntimes,len(zobs),nzo])

In [ ]:
for zi,zo in enumerate(zo_array):
    print('zo:',zo,'(',zi+1,'/',nzo,')')

    z = np.logspace(np.log10(zo),np.log10(zf),nz)
    dz = np.diff(z)
    
    w0_obs = np.interp(z,zobs[1:bini],wobs[0,1:bini])[1:-1]
    w0 = np.concatenate([np.real(w0_obs),
                           np.imag(w0_obs)])

    tic = np.datetime64('now')
    a = solve_ivp(dwdt_bbl, t_span=[tobs[0],tf], y0 = w0, t_eval = tobs[tobs<=tf], method='BDF')
    toc = np.datetime64('now')
    print(toc-tic)
    
    N = int(np.shape(a.y)[0]/2)
    t_sol = a.t
    w_sol = a.y[:N,:]+i*a.y[N:,:]
    
    fint= interp1d(z[1:-1],w_sol,axis=0,
                   bounds_error=False,fill_value = np.nan+i*np.nan)
    w_sol_obs = fint(zobs)
    
    w_sol_all[:,:,zi] = w_sol.T
    w_sol_obs_all[:,:,zi] = w_sol_obs.T

zo: 0.0001 ( 1 / 7 )
3402 seconds
zo: 0.00031622776601683794 ( 2 / 7 )


In [39]:
bbl_sols = xr.Dataset(
        data_vars={'uobs_subset':    (('tobs', 'zobs'), np.real(wobs[tobs<=tf,:])),
                   'vobs_subset':    (('tobs', 'zobs'), np.imag(wobs[tobs<=tf,:])),
                   'usol_all': (('tobs','zsol','zo'), np.real(w_sol_all)),
                   'vsol_all': (('tobs','zsol','zo'), np.imag(w_sol_all)),
                   'usol_obs_all': (('tobs','zobs','zo'), np.real(w_sol_obs_all)),
                   'vsol_obs_all': (('tobs','zobs','zo'), np.imag(w_sol_obs_all)),                  },
        coords={'zobs': zobs,
                'tobs': tobs[tobs<=tf],
                'zo':zo_array})
bbl_sols.to_netcdf('bbl_sols_newF.nc',mode='w')
bbl_sols.close()

In [ ]:
w_sol = np.squeeze(w_sol_all[:,:,0])

In [ ]:
plt.figure()
plt.subplot(211)
plt.plot(t_sol,np.real(w_sol),color='gray')
plt.plot(tobs[0:12],np.real(wobs[0:12,bini]))

plt.subplot(212)
plt.plot(t_sol,np.imag(w_sol),color='gray')
plt.plot(tobs[0:12],np.imag(wobs[0:12,bini]))

In [ ]:
plt.figure()
plt.plot(np.imag(np.squeeze(w_sol_obs_all[-1,:,:])),zobs)

In [ ]:
np.diff(a.t[0:100])

### Test with steady forcing

In [ ]:
winf_steady = 0.04+i*0
w0_steady = np.concatenate([np.real(winf_steady)*np.ones(N),
                           np.imag(winf_steady)*np.ones(N)])
zo = 0.01

def dwdt_bbl_steady(t, w_in):
    N = int(len(w_in)/2)
    w = w_in[:N]+i*w_in[N:]
    
    winf = winf_steady
    dwinfdt = 0+i*0

    F = dwinfdt + i*f*winf
    ustar = kappa*zo*np.abs(w[1])/dz[0]
    coef1 = 2*kappa*ustar/(dz[:-1]+dz[1:])

    wall = np.concatenate([np.atleast_1d(0+i*0),w,np.atleast_1d(winf)])
    
    dwalldt = np.nan*np.ones(np.shape(wall))+i*np.nan*np.ones(np.shape(wall))
    dwalldt[1:-1] = (F +
                     coef1*(z[1:-1]/dz[1:])*wall[2:] +
                     (-i*f - coef1*(z[1:-1]/dz[1:]+z[:-2]/dz[:-1]))*wall[1:-1] +
                     coef1*(z[:-2]/dz[:-1])*wall[:-2])
    
    dwdt = dwalldt[1:-1]
    #print(dwdt)
    
    dwdt_out = np.concatenate([np.real(dwdt),np.imag(dwdt)])
    
    return dwdt_out

In [ ]:
modsteady = solve_ivp(dwdt_bbl_steady, t_span=[0,2*86400], 
                      y0 = w0_steady, method='BDF')

In [ ]:
t_sol_steady = modsteady.t
w_sol_steady = modsteady.y[:N,:]+i*modsteady.y[N:,:]

In [ ]:
plt.figure()
plt.subplot(121)
plt.plot(np.real(w_sol_steady[:,-1]),z[1:-1]);
plt.plot(np.imag(w_sol_steady[:,-1]),z[1:-1]);
#plt.plot(tobs[0:12*24],wobs[0:12*24,bini])

In [ ]:
ustar_steady = kappa*zo*np.abs(w_sol_steady[1,-1])/dz[0]

In [ ]:
ustar_steady

In [ ]:
dz